# Importing libraries

In [ ]:

!pip install torch torchvision torchaudio
!pip install transformers requests beautifulsoup4 pandas numpy
!pip install datasets
!pip install kaggle

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datasets import Dataset
import requests
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




## Importing IMDB Dataset of 50K Movie Reviews to fine-tune the model with

### https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/code

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/iu_projects/kaggle.json"
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content/drive/MyDrive/iu_projects

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/drive/MyDrive/iu_projects/imdb-dataset-of-50k-movie-reviews.zip -d /content

Archive:  /content/drive/MyDrive/iu_projects/imdb-dataset-of-50k-movie-reviews.zip
replace /content/IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
print("Loading dataset...")
df = pd.read_csv("/content/drive/MyDrive/iu_projects/IMDB Dataset.csv")
# Convert sentiment to numeric labels
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to HuggingFace dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

Loading dataset...


In [ ]:
# Using entire dataset to train/test:
train_size = int(0.8 * len(train_dataset))  # Use ~80% for training
test_size = int(0.2 * len(test_dataset))    # Use ~20% for testing

train_dataset = train_dataset.shuffle(seed=42).select(range(train_size))
test_dataset = test_dataset.shuffle(seed=42).select(range(test_size))

In [ ]:
#Importing smaller Bert (looked for middle ground between Distilbert and tinyBert)
model_name = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["review"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

In [ ]:
train_dataset = train_dataset.rename_column("sentiment", "labels")
test_dataset = test_dataset.rename_column("sentiment", "labels")

# Tokenize the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch training
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

# Fine-tuning the model

In [ ]:
training_args = TrainingArguments(
    output_dir="./content/drive/MyDrive/iu_projects/results",
    num_train_epochs=3,                # Train for 3 epochs
    per_device_train_batch_size=64,    # Batch size 64
    per_device_eval_batch_size=64,
    eval_strategy="steps",             # Evaluate after each epoch
    save_strategy="steps",             # Save model after each epoch
    save_steps=200,
    eval_steps=200,
    learning_rate=1e-5,                #  1e-5
    weight_decay=0.01,                 # Help prevent overfitting
    load_best_model_at_end=True,       # Load the best model when training finishes
    report_to="none",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
print("Starting training...")
trainer.train()

#20 epochs, learning rate 2e-5 showed best result, but I have a life not a CPU resource

Starting training...


Step,Training Loss,Validation Loss,Accuracy
200,No log,0.533165,0.757500
400,No log,0.488450,0.777000
600,0.530200,0.466703,0.786500
800,0.530200,0.457942,0.794000
1000,0.455600,0.440223,0.804500
1200,0.455600,0.434434,0.809000
1400,0.455600,0.428466,0.811000


TrainOutput(global_step=1500, training_loss=0.4713410441080729, metrics={'train_runtime': 8261.7298, 'train_samples_per_second': 11.62, 'train_steps_per_second': 0.182, 'total_flos': 121966755840000.0, 'train_loss': 0.4713410441080729, 'epoch': 3.0})

In [ ]:
#Saving model
model.save_pretrained("/content/drive/MyDrive/iu_projects/imdb_sentiment_model")
tokenizer.save_pretrained("/content/drive/MyDrive/iu_projects_imdb_sentiment_model")

('/content/drive/MyDrive/iu_projects_imdb_sentiment_model/tokenizer_config.json',
 '/content/drive/MyDrive/iu_projects_imdb_sentiment_model/special_tokens_map.json',
 '/content/drive/MyDrive/iu_projects_imdb_sentiment_model/vocab.txt',
 '/content/drive/MyDrive/iu_projects_imdb_sentiment_model/added_tokens.json',
 '/content/drive/MyDrive/iu_projects_imdb_sentiment_model/tokenizer.json')

# Validation of the model with Rotten Tomatoes movies and critic reviews dataset  (or just split first dataset in 3 parts)
### https://www.kaggle.com/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset

In [ ]:
# Load the saved model and tokenizer
model_path = "/content/drive/MyDrive/iu_projects/imdb_sentiment_model"
tokenizer_path = "/content/drive/MyDrive/iu_projects_imdb_sentiment_model"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)


In [ ]:

# Load the Rotten Tomatoes dataset
print("Loading Rotten Tomatoes dataset for validation...")
file_path = "/content/drive/MyDrive/iu_projects/moviereviews.tsv"
rotten_tomatoes_df = pd.read_csv(file_path, sep='\t')

# Preprocess the data
# - Map 'neg' and 'pos' to numerical labels (0 and 1)
rotten_tomatoes_df['labels'] = rotten_tomatoes_df['label'].map({'neg': 0, 'pos': 1})

# Data cleaning:
# Replace NaN values with empty strings
rotten_tomatoes_df['review'] = rotten_tomatoes_df['review'].fillna('')
# Convert 'review' column to string type
rotten_tomatoes_df['review'] = rotten_tomatoes_df['review'].astype(str)

# 3. Convert to Hugging Face dataset format
rotten_tomatoes_dataset = Dataset.from_pandas(rotten_tomatoes_df)

# 4. Tokenize the dataset (using your previously loaded tokenizer)
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=512)

# Tokenize each row individually instead of .map
tokenized_rotten_tomatoes = rotten_tomatoes_dataset.map(tokenize_function, batched=False)

tokenized_rotten_tomatoes.set_format("torch", columns=["input_ids", "attention_mask", "labels"])



Loading Rotten Tomatoes dataset for validation...


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Computing the metrics
predictions = []
labels = []

for batch in tokenized_rotten_tomatoes:
    input_ids = batch['input_ids'].unsqueeze(0)  # batch dimension
    attention_mask = batch['attention_mask'].unsqueeze(0)
    with torch.no_grad():  # disable gradient calculation during inference
           outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_class = logits.argmax(-1).item()
    predictions.append(predicted_class)
    labels.append(batch['labels'].item())

In [ ]:
#using some more metrics because i didnt take 2 stats classes for nothing
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
f1 = f1_score(labels, predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.741
Precision: 0.7875894988066826
Recall: 0.66
F1-score: 0.7181719260065288


#Scraping Web

In [ ]:
# Scrape movie reviews from Rotten Tomatoes
r = requests.get('https://www.rottentomatoes.com/m/paddington_in_peru/reviews')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*review-text.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [ ]:
#optional to try first 20 reviews first, comment out later
# Preprocess and tokenize the reviews

# Checking reviews line by line, first 20 to verify that it works ok
inputs = tokenizer(reviews, padding=True, truncation=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(-1).tolist()

# Output the predictions
for i, prediction in enumerate(predictions):
    print(f"Review {i + 1}: Sentiment = {prediction}")  # 0 for negative, 1 for positive

Review 1: Sentiment = 0
Review 2: Sentiment = 1
Review 3: Sentiment = 1
Review 4: Sentiment = 1
Review 5: Sentiment = 1
Review 6: Sentiment = 1
Review 7: Sentiment = 0
Review 8: Sentiment = 1
Review 9: Sentiment = 1
Review 10: Sentiment = 1
Review 11: Sentiment = 1
Review 12: Sentiment = 0
Review 13: Sentiment = 1
Review 14: Sentiment = 1
Review 15: Sentiment = 1
Review 16: Sentiment = 1
Review 17: Sentiment = 1
Review 18: Sentiment = 1
Review 19: Sentiment = 1
Review 20: Sentiment = 1


In [ ]:
# Scrape movie reviews from Rotten Tomatoes, IMDB, BBC, Reddit to know whether I'm watching "Paddington in Peru"
this weekendr = requests.get('https://www.rottentomatoes.com/m/paddington_in_peru/reviews')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*review-text.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

# --- Add IMDB scraping ---
imdb_url = 'https://www.imdb.com/title/tt5822536/reviews/?ref_=ttrt_sa_3'
imdb_response = requests.get(imdb_url)
imdb_soup = BeautifulSoup(imdb_response.text, 'html.parser')

imdb_results = imdb_soup.find_all('div', {'class': 'ipc-html-content-inner-div'})
for result in imdb_results:

    review_text = result.text.strip()
    reviews.append(review_text)

# --- End IMDB scraping ---


# --- Add BBC scraping ---
bbc_url = 'https://www.bbc.com/culture/article/20241103-paddington-in-peru-review-paddington-2'
bbc_response = requests.get(bbc_url)
bbc_soup = BeautifulSoup(bbc_response.text, 'html.parser')

bbc_results = bbc_soup.find_all('div', {'class': 'text-block'})
for result in bbc_results:
    # Extract the text content of the review
    # Assuming the review text is directly within the 'text-block' div
    review_text = result.get_text(strip=True)
    reviews.append(review_text)

# --- End BBC scraping ---


# --- Add Reddit scraping ---
reddit_url = 'https://www.reddit.com/r/movies/comments/1ip0uph/official_discussion_paddington_in_peru_spoilers/'
reddit_response = requests.get(reddit_url, headers={'User-agent': 'your bot 0.1'})
reddit_soup = BeautifulSoup(reddit_response.content, 'html.parser')

reddit_results = reddit_soup.find_all('div', {'class': 'userContentWrapper'})
for result in reddit_results:

    script_tag = result.find('script', type='application/ld+json')
    if script_tag:
        try:
            json_data = json.loads(script_tag.string)
            review_text = json_data['text']
            reviews.append(review_text)
        except (json.JSONDecodeError, KeyError):
            pass


# --- End Reddit scraping ---


# Preprocess and tokenize the reviews
inputs = tokenizer(reviews, padding=True, truncation=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(-1).tolist()

# Calculate overall sentiment
positive_count = sum(predictions)
negative_count = len(predictions) - positive_count

if positive_count > negative_count:
    overall_sentiment = "Positive"
elif positive_count < negative_count:
    overall_sentiment = "Negative"
else:
    overall_sentiment = "Neutral"

print(f"Overall Sentiment: {overall_sentiment}")

Overall Sentiment: Positive


### Overall sentiment is positive, we're watching the new Paddigton!

